# Title

In [ ]:
%config InteractiveShell.ast_node_interactivity='last_expr_or_assign'  # always print last expr.
%config InlineBackend.figure_format = 'svg'
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import logging
from abc import ABC, abstractmethod
from typing import Any, Literal, Optional, Union
from collections import defaultdict

import torch
import numpy as np
import pandas.api.types
from pandas import (
    DatetimeIndex,
    Series,
    Timedelta,
    Timestamp,
    DataFrame,
    Index,
    MultiIndex,
    NA,
)
from torch import Tensor

In [ ]:
from tsdm.datasets import ETTh1

ds = ETTh1.dataset

## Standardizer

In [ ]:
from tsdm.encoders import Standardizer

encoder = Standardizer()
encoder.fit(ds)
encoded = encoder.encode(ds)

In [ ]:
decoded = encoder.decode(ds)

## ChainedEncoder

In [ ]:
enc = Standardizer() @ Standardizer()

In [ ]:
enc.fit(ds)
enc[0].mean, enc[1].mean

In [ ]:
encoder = Standardizer()
encoder.fit(ds)
encoded = encoder.encode(ds)

In [ ]:
decoded = encoder.decode(ds)

## DataFrameEncoder

In [ ]:
DataFrame(ds["OT"]).dtypes

In [ ]:
from tsdm.encoders import DataFrameEncoder, Standardizer, DateTimeEncoder
from tsdm.datasets import ETTh1

ds = ETTh1.dataset

encoderA = Standardizer()
encoderB = Standardizer()
encoderC = Standardizer()

encoders = {
    "HUFL": encoderA,
    "HULL": encoderB,
    "MUFL": encoderA,
    "MULL": encoderC,
    "LUFL": encoderB,
    "LULL": encoderB,
    "OT": encoderB,
}

In [ ]:
e = DataFrameEncoder(encoders, index_encoder=DateTimeEncoder("h"))

In [ ]:
e.fit(ds)
encoded = e.encode(ds)
decoded = e.decode(encoded)
pandas.testing.assert_frame_equal(ds, decoded)

### DataFrameEncoder Test Implementation

In [ ]:
from tsdm.encoders import BaseEncoder


class DataFrameEncoder:
    r"""Combine multiple encoders into a single one.

    It is assumed that the DataFrame Modality doesn't change.
    """

    column_encoder: Union[BaseEncoder, dict[Any, BaseEncoder]]
    r"""Encoders for the columns."""
    index_encoder: Optional[BaseEncoder] = None
    r"""Optional Encoder for the index."""
    colspec: list[str] = None
    r"""The columns-specification of the DataFrame."""
    encode_index: bool
    r"""Whether to encode the index."""
    column_wise: bool
    r"""Whether to encode column-wise"""
    partitions: Optional[dict] = None
    r"""Contains partitions if used column wise"""

    def __init__(
        self,
        encoders: Union[BaseEncoder, dict[Any, BaseEncoder]],
        *,
        index_encoder: Optional[BaseEncoder] = None,
    ):
        r"""Set up the individual encoders.

        Note: the same encoder instance can be used for multiple columns.

        Parameters
        ----------
        encoders
        index_encoder
        """
        super().__init__()
        self.column_encoder = encoders
        self.index_encoder = index_encoder
        self.column_wise: bool = isinstance(self.column_encoder, dict)
        self.encode_index: bool = index_encoder is not None

        if self.encode_index:
            _idxenc_spec = {
                "col": NA,
                "encoder": self.index_encoder,
                "dim_in": NA,
                "dim_out": NA,
            }
            idxenc_spec = DataFrame.from_records(
                _idxenc_spec, index=Index([NA], name="partition")
            )
        else:
            idxenc_spec = DataFrame(
                columns=["col", "encoder", "dim_in", "dim_out"],
                index=Index([], name="partition"),
            )

        if not self.column_wise:
            _colenc_spec = {
                "col": NA,
                "encoder": self.column_encoder,
                "dim_in": NA,
                "dim_out": NA,
            }
            colenc_spec = DataFrame.from_records(
                _colenc_spec, index=Index([0], name="partition")
            )
        else:
            keys = self.column_encoder.keys()
            assert len(set(keys)) == len(keys), f"Some keys are duplicates!"

            _encoders = tuple(set(self.column_encoder.values()))
            encoders = Series(_encoders, name="encoder")
            partitions = Series(range(len(_encoders)), name="partition")

            _columns = defaultdict(list)
            for key, encoder in self.column_encoder.items():
                _columns[encoder].append(key)

            columns = Series(_columns, name="col")

            colenc_spec = DataFrame(encoders, index=partitions)
            colenc_spec = colenc_spec.join(columns, on="encoder")
            colenc_spec["dim_in"] = colenc_spec["col"].apply(len)
            colenc_spec["dim_out"] = pandas.NA

        self.spec = pandas.concat(
            [idxenc_spec, colenc_spec],
            keys=["index", "columns"],
            names=["section", "partition"],
        )

In [ ]:
e = DataFrameEncoder(encoders, index_encoder=Standardizer())
e.spec

In [ ]:
e.spec.loc["index", "dim_out"] = 3
torch.randn(3, 4, 5)[..., e.spec.loc["index", "dim_out"].item()].shape

In [ ]:
e.spec.loc["index", "dim_out"].item()

In [ ]:
torch.Tensor(ds.values).nanmean(axis=None)

In [ ]:
torch.mean(torch.tensor([float("nan"), 2]))

In [ ]:
np.nanstd

In [ ]:
torch.nanmean(torch.Tensor(ds.values), dim=None)

In [ ]:
torch.Tensor(ds.values).nanmean(dim=None)

In [ ]:
ds["OT"].mean()

In [ ]:
indexenc = DataFrame.from_records(
    {"Encoders": object(), "dim_in": 1, "dim_out": 5}, index=Index([0])
)

In [ ]:
DataFrame.from_dict({"Encoders": object(), "dim_in": 1, "dim_out": 5}, orient="index").T

In [ ]:
pandas.concat(
    [indexenc, e.spec], keys=["index", "columns"], names=["section", "partiton"]
)

In [ ]:
idx = MultiIndex(
    levels=[["index", "columns"], []], codes=[[], []], names=["section", "partition"]
)

In [ ]:
spec = DataFrame(
    columns=["col", "encoder", "dim_in", "dim_out"],
    index=Index([], name="partition")
    # index=MultiIndex(
    #     levels=[["index", "columns"], []],
    #     codes=[[], []],
    #     names=["section", "partition"],
    # ),
)

In [ ]:
_index_encoder_spec = {
    "col": NA,
    "encoder": object(),
    "dim_in": NA,
    "dim_out": NA,
}
index_encoder_spec = DataFrame.from_records(
    _index_encoder_spec, index=Index([0], name="partition")
)

## Concat Encoder

## Time2Float

## DateTimeEncoder

### Combined

In [ ]:
from tsdm.encoders import (
    DateTimeEncoder,
    DataFrameEncoder,
    IdentityEncoder,
    TensorEncoder,
    Standardizer,
    FloatEncoder,
)

In [ ]:
df_encoder = DataFrameEncoder(
    (Standardizer() @ FloatEncoder()), index_encoder=DateTimeEncoder()
)

Tensorizer = TensorEncoder()

encoder = TensorEncoder() @ df_encoder

In [ ]:
from tsdm.datasets import ETTh1

ds = ETTh1.dataset

In [ ]:
enc = Standardizer() @ FloatEncoder()
enc.fit(ds)

In [ ]:
enc.encode(ds)

In [ ]:
encoder.fit(ds)

In [ ]:
encoded = encoder.encode(ds)

In [ ]:
decoded = encoder.decode(encoded)

In [ ]:
modified = encoded[0] * 2, encoded[1] + 1
encoder.decode(modified)